In [7]:
# Disable info messages from the tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D,LSTM,Embedding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom
import pickle

In [2]:
############### Load the dataset ###################
path = 'connect-4.csv'
classes = 3
dataset = np.genfromtxt("connect-4.csv", dtype='str', delimiter=",")
X = dataset[:,0:42]
Y = dataset[:,42]
#Y = LabelEncoder().fit_transform(Y)
#Y = to_categorical(Y,classes)
print(X.shape,Y.shape)

(67557, 42) (67557,)


In [3]:
x = np.zeros(X.shape)

for i, row in enumerate(X):
    for j, col in enumerate(row):
        if col == 'x':
            x[i,j] = 1.0
        if col == 'o':
            x[i,j] = -1.0
        if col == 'b':
            x[i,j] = 0.0

Y = LabelEncoder().fit_transform(Y)
x = MinMaxScaler().fit_transform(x)
print(x.shape,Y.shape)

(67557, 42) (67557,)


In [4]:
X_train,X_test, Y_train,Y_test = train_test_split(x,Y,test_size=0.259884)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(50000, 42) (50000,)
(17557, 42) (17557,)


In [9]:
model = Sequential()
model.add(Embedding(50000, 5000, input_length=42))
model.add(LSTM(20, dropout=0.2,activation= 'tanh',return_sequences=True))
model.add(Dense(classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,Y_train,shuffle=True,batch_size=256,epochs=20,validation_data=(X_test,Y_test))

Epoch 1/20


ValueError: in user code:

    c:\users\valan\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    c:\users\valan\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\valan\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\valan\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\valan\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:532 train_step  **
        loss = self.compiled_loss(
    c:\users\valan\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\keras

In [ ]:
score = model.evaluate(X_test,Y_test)
print(score[1])